In [1]:
# ResNet：深度残差网络
# http://zh.gluon.ai/chapter_convolutional-neural-networks/resnet-gluon.html

from mxnet.gluon import nn
from mxnet import nd
import utils

class _Residual(nn.Block):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(_Residual, self).__init__(**kwargs)
        self.same_shape = same_shape
        strides = 1 if same_shape else 2
        self.conv1 = nn.Conv2D(channels, kernel_size=3, padding=1,
                              strides=strides)
        self.bn1 = nn.BatchNorm()
        self.conv2 = nn.Conv2D(channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm()
        if not same_shape:
            self.conv3 = nn.Conv2D(channels, kernel_size=1,
                                  strides=strides)
            
class ResidualIdentity(_Residual):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(ResidualIdentity, self).__init__(channels, same_shape, **kwargs)
    
    def forward(self, x):
#         print('x.shape:', x.shape)
        
        out = self.conv1(nd.relu(self.bn1(x)))
#         print('out.shape:', out.shape)
        
        out = self.conv2(nd.relu(self.bn2(out)))
#         print('out.shape:', out.shape)
        
        if not self.same_shape:
            x = self.conv3(x)
#             print('x.shape:', x.shape)
            
        return out + x

/home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 输入输出通道相同
blk = ResidualIdentity(3)
blk.initialize()

x = nd.random.uniform(shape=(4,3,96,96))
y = blk(x)
print('y.shape:', y.shape)
print(blk)

y.shape: (4, 3, 96, 96)
ResidualIdentity(
  (conv1): Conv2D(3 -> 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=3)
  (conv2): Conv2D(3 -> 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=3)
)


In [3]:
# 输入输出通道不同
blk2 = ResidualIdentity(8, same_shape=False)
blk2.initialize()
# print(blk2)
y2 = blk2(x)
print('y2.shape:',y2.shape)
print(blk2)

y2.shape: (4, 8, 48, 48)
ResidualIdentity(
  (conv1): Conv2D(3 -> 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=3)
  (conv2): Conv2D(8 -> 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=8)
  (conv3): Conv2D(3 -> 8, kernel_size=(1, 1), stride=(2, 2))
)


In [14]:
class _ResidualBottleneck(nn.Block):
    def __init__(self, channels_in, channels_out, same_shape=True, **kwargs):
        super(_ResidualBottleneck, self).__init__(**kwargs)
        self.same_shape = same_shape
        strides = 1 if same_shape else 2
        self.conv1 = nn.Conv2D(channels_in, kernel_size=1,
                              strides=strides)
        self.bn1 = nn.BatchNorm()
        self.conv2 = nn.Conv2D(channels_in, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm()
        self.conv3 = nn.Conv2D(channels_out, kernel_size=1,
                              strides=1)
        self.bn3 = nn.BatchNorm()
        if not same_shape:
            self.conv4 = nn.Conv2D(channels_out, kernel_size=1,
                                  strides=strides)
            
#     def forward(self, x):
# #         print('same in / out shape:', self.same_shape)
# #         print('x.shape:', x.shape)
#         out = nd.relu(self.bn1(self.conv1(x)))
# #         print(out.shape)
#         out = nd.relu(self.bn2(self.conv2(out)))
# #         print(out.shape)
#         out = self.bn3(self.conv3(out))
# #         print(out.shape)
#         if not self.same_shape:
#             x = self.conv4(x)
# #             print('x.shape:',x.shape)
#         return nd.relu(out + x)
class ResidualIdentityBottleneck(_ResidualBottleneck):
    def __init__(self, channels_in, channels_out, same_shape=True, **kwargs):
        super(ResidualIdentityBottleneck, self).__init__(channels_in, channels_out, same_shape, **kwargs)
    
    def forward(self, x):
        print('same_shape:', self.same_shape)
        print('x.shape:', x.shape)

        out = self.conv1(nd.relu(self.bn1(x)))
        print('out.shape:', out.shape)

        out = self.conv2(nd.relu(self.bn2(out)))
        print('out.shape:', out.shape)

        out = self.conv3(nd.relu(self.bn3(out)))
        print('out.shape:', out.shape)

        if not self.same_shape:
            x = self.conv3(x)
            print('x.shape:', x.shape)

        return out + x

In [16]:
# 构建ResNet
class ResNet50(nn.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super(ResNet50, self).__init__(**kwargs)
        self.verbose = verbose
        # add name_scope on the outermost Sequential
        with self.name_scope():
            # block 1
            b1 = nn.Conv2D(64, kernel_size=7, strides=2)
            # block 2
            b2 = nn.Sequential()
            b2.add(
                nn.MaxPool2D(pool_size=3, strides=2),
                ResidualIdentityBottleneck(64, 256, same_shape=False),
                ResidualIdentityBottleneck(64, 256),
                ResidualIdentityBottleneck(64, 256)
            )
            # block 3
            b3 = nn.Sequential()
            b3.add(
                ResidualIdentityBottleneck(128, 512, same_shape=False),
                ResidualIdentityBottleneck(128, 512),
                ResidualIdentityBottleneck(128, 512),
                ResidualIdentityBottleneck(128, 512)
            )
            # block 4
            b4 = nn.Sequential()
            b4.add(
                ResidualIdentityBottleneck(256, 1024, same_shape=False),
                ResidualIdentityBottleneck(256, 1024),
                ResidualIdentityBottleneck(256, 1024),
                ResidualIdentityBottleneck(256, 1024),
                ResidualIdentityBottleneck(256, 1024),
                ResidualIdentityBottleneck(256, 1024)
            )
            # block 5
            b5 = nn.Sequential()
            b5.add(
                ResidualIdentityBottleneck(512, 2048, same_shape=False),
                ResidualIdentityBottleneck(512, 2048),
                ResidualIdentityBottleneck(512, 2048)
            )
            # block 6
            b6 = nn.Sequential()
            b6.add(
                nn.AvgPool2D(pool_size=3),
                nn.Dense(num_classes)
            )
            # chain all blocks together
            self.net = nn.Sequential()
            self.net.add(b1, b2, b3, b4, b5, b6)
    
    def forward(self, x):
        out = x
        for i, b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('Block %d output: %s' % (i+1, out.shape))
        return out

In [17]:
net = ResNet50(10, verbose=True)
net.initialize()

x = nd.random.uniform(shape=(4,3,224, 224))
y = net(x)
print(net)

Block 1 output: (4, 64, 109, 109)
same_shape: False
x.shape: (4, 64, 54, 54)
out.shape: (4, 64, 27, 27)
out.shape: (4, 64, 27, 27)
out.shape: (4, 256, 27, 27)
x.shape: (4, 256, 54, 54)


MXNetError: [09:43:41] src/operator/tensor/./elemwise_binary_broadcast_op.h:68: Check failed: l == 1 || r == 1 operands could not be broadcast together with shapes [4,256,27,27] [4,256,54,54]

Stack trace returned 10 entries:
[bt] (0) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x3032da) [0x7f30082ee2da]
[bt] (1) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x303901) [0x7f30082ee901]
[bt] (2) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xba6477) [0x7f3008b91477]
[bt] (3) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x24e05ba) [0x7f300a4cb5ba]
[bt] (4) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x24ea589) [0x7f300a4d5589]
[bt] (5) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x241c2a9) [0x7f300a4072a9]
[bt] (6) /home/ly/anaconda3/envs/learning/lib/python3.6/site-packages/mxnet

In [ ]:
# 读取数据
import sys
sys.path.append('..')
import utils
from mxnet import autograd
from mxnet import gluon
from mxnet import nd
from mxnet import image
from mxnet import init

batch_size = 128
resize=224
learning_rate = 0.01

train_data, test_data = utils.load_data(batch_size=batch_size, resize=resize)

ctx = utils.try_gpu(1)
net = ResNet50(10, verbose=False)
net.initialize(ctx=ctx, init=init.Xavier())

In [ ]:
utils.train(net=net, train_data=train_data, test_data=test_data,ctx=ctx,
            batch_size=batch_size, learning_rate=learning_rate)